In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained(
    "veliander/autonlp-dta15-32947883", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained(
    "veliander/autonlp-dta15-32947883", use_auth_token=True)


Downloading:   0%|          | 0.00/848 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
import pandas as pd
# Load test data
test = pd.read_csv('test.csv')
X_test = test['Transcription']
y_test = test['Type'] == 'Truthful'


inputs = tokenizer(X_test[0], return_tensors="pt")
outputs = model(**inputs)
print(outputs)  

#print(X_test[0])


SequenceClassifierOutput(loss=None, logits=tensor([[-0.1011,  0.4673]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
import torch
import numpy as np

m = nn.Softmax(dim=1)
o = m(outputs.logits)


for doc in X_test:
    #inputs = tokenizer(doc, return_tensors="pt")
    #outputs = model(**inputs)
    o = m(outputs.logits)
    print(f"{o.data[0][0]:.2f} Truthful") 



